# Train and Deploy model

Prepare data, train XGBoost model, and deploy as real time endpoint to sagemaker.

## Data Preparation and Training

In [1]:
# Step 1: Setup
import sagemaker
from sagemaker import get_execution_role
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split 

sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name

# S3 bucket for storing data
bucket = 'kennys-testing-bucket'
prefix = 'deployment'
output_path = f's3://{bucket}/{prefix}/output'

# Load the dataset
file_path = 'Employee.csv'  # Replace with your actual file path in S3 if needed
employee_df = pd.read_csv(file_path)
employee_df.head()

# Step 2: Data Preparation
# Convert categorical columns to numeric
employee_df['Education'] = employee_df['Education'].astype('category').cat.codes
employee_df['City'] = employee_df['City'].astype('category').cat.codes
employee_df['Gender'] = employee_df['Gender'].astype('category').cat.codes
employee_df['EverBenched'] = employee_df['EverBenched'].map({'Yes': 1, 'No': 0})

# Drop rows with NaN values in the target column
employee_df.dropna(subset=['LeaveOrNot'])

# Convert target column to numeric if needed
employee_df['LeaveOrNot'] = employee_df['LeaveOrNot'].astype(int)

# Ensure no missing values in feature columns
employee_df = employee_df.dropna()

# Verify all columns are numeric
print(employee_df.dtypes)

# Define features and target
feature_columns = [
    'Education', 'JoiningYear', 'City', 'PaymentTier', 'Age',
    'Gender', 'EverBenched', 'ExperienceInCurrentDomain'
]
target_column = 'LeaveOrNot'

employee_df = employee_df[[target_column] + feature_columns]

train_df, test_df = train_test_split(employee_df, test_size=0.2, random_state=42)

# Display the transformed dataset
employee_df.head()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Education                     int8
JoiningYear                  int64
City                          int8
PaymentTier                  int64
Age                          int64
Gender                        int8
EverBenched                  int64
ExperienceInCurrentDomain    int64
LeaveOrNot                   int64
dtype: object


,LeaveOrNot,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
0,0,0,2017,0,3,34,1,0,0
1,1,0,2013,2,1,28,0,0,3
2,0,0,2014,1,3,38,0,0,2
3,1,1,2016,0,3,27,1,0,5
4,1,1,2017,2,3,24,1,1,2


In [5]:
from sagemaker.inputs import TrainingInput

# Initialize S3 client
s3 = boto3.client('s3')

# Save the data locally first
train_file = 'train.csv'
validation_file = 'validation.csv'
train_df.to_csv(train_file, index=False)
test_df.to_csv(validation_file, index=False)

# Upload the data to S3
s3.upload_file(train_file, bucket, f'{prefix}/train/{train_file}')
s3.upload_file(validation_file, bucket, f'{prefix}/validation/{validation_file}')

print(f"Training data uploaded to s3://{bucket}/{prefix}/train/{train_file}")
print(f"Validation data uploaded to s3://{bucket}/{prefix}/validation/{validation_file}")

# Setup XGBoost Estimator
xgboost_container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.3-1")
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"40",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",
    "num_round":"50"
}

output_path = f's3://{bucket}/{prefix}/output'

estimator = sagemaker.estimator.Estimator(
    image_uri=xgboost_container, 
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1, 
    instance_type='ml.m5.xlarge', 
    volume_size=5,  # 5 GB 
    output_path=output_path
)

# Define the data type and paths to the training and validation datasets
content_type = "csv"
train_input = TrainingInput(f"s3://{bucket}/{prefix}/train/{train_file}", content_type=content_type)
validation_input = TrainingInput(f"s3://{bucket}/{prefix}/validation/{validation_file}", content_type=content_type)

# Execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})

Training data uploaded to s3://kennys-testing-bucket/deployment/train/train.csv
Validation data uploaded to s3://kennys-testing-bucket/deployment/validation/validation.csv


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-12-30-20-01-36-049


2024-12-30 20:01:38 Starting - Starting the training job...
2024-12-30 20:01:52 Starting - Preparing the instances for training...
2024-12-30 20:02:32 Downloading - Downloading the training image...
2024-12-30 20:03:08 Training - Training image download completed. Training in progress...[2024-12-30 20:03:18.074 ip-10-0-141-69.us-east-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-12-30 20:03:18.101 ip-10-0-141-69.us-east-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-12-30:20:03:18:INFO] Imported framework sagemaker_xgboost_container.training
[2024-12-30:20:03:18:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-12-30:20:03:18:INFO] No GPUs detected (normal if no gpus installed)
[2024-12-30:20:03:18:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-12-30:20:03:18:INFO] Determined delimiter of CSV input is ','
[2024-12-30:20:03:18:INFO] Determi

## Deploy as Real-Time Endpoint

In [6]:

# Deploy the model as a real-time endpoint
predictor = estimator.deploy(
    initial_instance_count=1,  # Number of instances to deploy
    instance_type='ml.m5.xlarge',  # Instance type for the endpoint
    endpoint_name='xgboost-predictor-realtime'  # Name of the endpoint
)


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-12-30-20-04-35-095
INFO:sagemaker:Creating endpoint-config with name xgboost-predictor-endpoint
INFO:sagemaker:Creating endpoint with name xgboost-predictor-endpoint


-----!

### Alternatively deploy from a model artifact

In [ ]:
from sagemaker.model import Model

# Specify the S3 path to the pre-trained model artifact
model_artifact = "s3://your-path/model.tar.gz"

# Retrieve the container image for the framework (e.g., XGBoost)
container = sagemaker.image_uris.retrieve(framework="xgboost", region=boto3.Session().region_name, version="1.3-1")

# Create the model object using the S3 path
model = Model(
    image_uri=container,
    model_data=model_artifact,
    role=sagemaker.get_execution_role()
)

# Deploy the model as a real-time endpoint
predictor = model.deploy(
    initial_instance_count=1,  # Number of instances
    instance_type='ml.m5.xlarge',  # Instance type
    endpoint_name='xgboost-predictor-realtime'  # Name of the endpoint
)



## Prepare data for inference

In [7]:

# Optionally, configure the predictor for the specific input and output formats
predictor.serializer = sagemaker.serializers.CSVSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

# Example input data (in the same format as your training data)

# Assuming 'LeaveOrNot' is the target column
features_df = test_df.drop(columns=['LeaveOrNot'])

# Convert the features DataFrame to CSV format
test_csv = features_df.to_csv(index=False, header=False).strip()

test_data = test_df[feature_columns].head(20)  # Select the first row of test data for prediction
test_data_csv = test_data.to_csv(index=False, header=False).strip()  # Convert to CSV format

: 

## Invoke endpoint by finding the endpoint at runtime

In [36]:
import boto3

# Initialize the SageMaker runtime client
runtime_client = boto3.client('sagemaker-runtime')

# Specify the endpoint name
endpoint_name = 'xgboost-predictor-realtimer'

# Invoke the endpoint directly using the runtime client
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",  # Specify the content type
    Body=test_data_csv  # The input data as a CSV string
)

# Parse the response
result = response['Body'].read().decode('utf-8')
print(result)


0.20906561613082886
0.1683102250099182
0.38488397002220154
0.4326663315296173
0.15928953886032104
0.46319764852523804
0.6720890998840332
0.9404475688934326
0.15928953886032104
0.9504724144935608
0.12999935448169708
0.20906561613082886
0.9239230155944824
0.12999935448169708
0.2676047682762146
0.9504724144935608
0.15928953886032104
0.42358148097991943
0.47207778692245483
0.12999935448169708
0.15928953886032104
0.5771342515945435
0.709308922290802
0.1683102250099182
0.3065555989742279

